In [ ]:
import pandas as pd

import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set_style("whitegrid")

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"]) & 
    (df["subcategory"] == "Cost per MWh delivered")]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "No flexibility",    
    "daily"     : "Daily",
    "weekly"    : "Weekly",
    "biweekly"  : "Biweekly",
    "monthly"   : "Month",
    "quaterly"  : "Quaterly",
    "annually"  : "Yearly",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility,
    categories=flexibilities.values(),
    ordered=True)

df = df.sort_values("flexibility")

# Plot
fig, ax = plt.subplots(figsize=(6,3))

# First plot pipeline ESC
esc = "pipeline-h2-to-elec"
for label, plot_df in df.query("esc == @esc").groupby("csp"):
    
    text = "H2 pipeline (w/   CSP, TES)" if label else "H2 pipeline (w/o CSP, TES)"
    plot_df.plot(
        x="flexibility", y="value", ax=ax,
        marker="o",
        label=text,
        fillstyle="full",
        markerfacecolor="green" if label else "white",
        color="green",
    )
    
# second plot HVDC ESC
esc = "hvdc-to-elec"
for label, plot_df in df.query("esc == @esc").groupby("csp"):
    
    text = "HVDC (w/   CSP, TES)" if label else "HVDC (w/o CSP, TES)"
    plot_df.plot(
        x="flexibility", y="value", ax=ax,
        marker="*",
        markersize=9,
        label=text,
        fillstyle="full",
        markerfacecolor="black" if label else "white",
        color="black",
    )

ax.set_ylabel("Electricity cost [EUR/MWh]")
ax.set_xlabel("Flexibility to shift load")
ax.set_ylim(0,205) # Magic number 200: Highest value from TN ESC H2 pipeline

handles, labels = ax.get_legend_handles_labels()

# hard coded, intended to be
#H2 pipe (with CSP)
#H2 pipe (without CSP)
# HVDC (with CSP)
# HVDC (without CSP)
new_order = [1,0,3,2]

ax.legend([handles[idx] for idx in new_order], [labels[idx] for idx in new_order],
           loc="lower left", ncols=2)

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches='tight')
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches='tight')